In [4]:
import os
import subprocess
import csv

In [2]:
#Create List of .bam files

bams = []

for file in os.listdir('.'):
    if file.endswith('.bam'):
        bams.append(file)

In [3]:
#Convert full bam to fastq format 

convert = str('samtools fastq '+bams[0]+' > raw_seqs.fastq')
subprocess.run(convert, shell=True)

[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 451347 reads


CompletedProcess(args='samtools fastq subsample.bam > raw_seqs.fastq', returncode=0)

In [4]:
Fwd_bcs = {'01':'CTAAGGTAACGAT', 
           '02':'TAAGGAGAACGAT', 
           '03':'AAGAGGATTCGAT', 
           '04':'TCGCAATTACGAT',
           '05':'CAGAAGGAACGAT',
           '06':'CTGCAAGTTCGAT',
           '07':'TTCGTGATTCGAT',
           '08':'TTCCGATAACGAT',
           '09':'TGAGCGGAACGAT',
           '10':'CTGACCGAACGAT',}

f = open('fwd_sort.txt', 'w')
for k in Fwd_bcs:
    fwd_sort = str('cutadapt -g '+Fwd_bcs[k]+' --discard-untrimmed -e 0 -j 0 --overlap 10 -o F'
               +k+'.fastq raw_seqs.fastq')
    subprocess.run(fwd_sort, shell=True, stdout=f)

os.system('find . -type f -size 0 -delete')

0

In [5]:
#Running cutadapt for the reverse compliment of each REV index
    #Trim past 3 prime end and save into new file based on F+R barcodes 
    
Rev_bcs = {'11':'GATTCGAGGA', 
           '12':'GAACCACCTA', 
           '13':'GTCCGTTAGA', 
           '14':'GACACTCCAA',
           '15':'GACCTCTAGA',
           '16':'GTCATCCAGA',
           '17':'GACGAATAGA',
           '18':'GCAATTGCCT',
           '19':'GTCCGACTAA',
           '20':'GATGGATCTG'}

fastqs = []
for file in os.listdir('.'):
    if len(file) == 9 and file.endswith('fastq'): 
        fastqs.append(file)

f = open('rev_sort.txt', 'w')
for fastq in fastqs:
    for k in Rev_bcs:
        rev_sort = str('cutadapt -a '+Rev_bcs[k]+' --discard-untrimmed -e 0 -j 0 --overlap 10 -o '
                       +fastq[0:3]+'R'+k+'.fastq '+fastq)
        subprocess.run(rev_sort, shell=True, stdout=f)
        
os.system('find . -type f -size 0 -delete')

0

In [6]:
#'Re-Multiplex', or combine the reverse index (trimmed off above) with the original forward index
# creating a single 20bp barcode at the beginning of each read
# also concatenates everything into one 'All_Seqs.fastq'

#Dictionary of all ID & 20bp index/barcode combos 
reader = csv.reader(open('20bp_bcs_1-20.csv', 'r', encoding='utf-8-sig'))
bcs_20bp = {}
for row in reader:
    k, v = row
    bcs_20bp[k] = v 

    fastqs = []
for fastq in os.listdir('.'):
    if len(fastq) == 12 and fastq.endswith('fastq'):
        fastqs.append(fastq)
        
fastqs.sort()


for fastq in fastqs:
    for k ,v in bcs_20bp.items():
        if fastq[0:6] == k:
            with open(fastq, 'r') as f, open('all_seqs.fastq', 'a+') as out:
                count = 0
                for idx, line in enumerate(f.read().splitlines()):
                    count += 1
                    if count == 2:
                        print(v+line, file=out)
                    elif count == 4:
                        print(''.join(['I']*len(v))+line, file=out)
                    else:
                        print(line, file=out)
                    if count == 4:
                        count = 0
        else:
            continue

In [49]:
#Prepare for Qiime2 Import            
#os.system('gzip all_seqs.fastq')

0

In [7]:
#Clean-up Steps
os.system('mkdir fastqs')
os.system('mv F*.fastq fastqs| mv fwd_sort.txt fastqs| mv rev_sort.txt fastqs | rm raw_seqs.fastq')

0